<a href="https://colab.research.google.com/github/romanozamoth/transfer-learning-ml-dio/blob/main/New_model_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch torchvision scikit-learn pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch
import os
import numpy as np
from sklearn.linear_model import LogisticRegression
import joblib


In [ ]:
from zipfile import ZipFile
from google.colab import files

# Subir o seu dataset.zip com os modelos que quer treinar
uploaded = files.upload()

# Supondo que você fez upload de dataset.zip com estrutura de pastas
with ZipFile("dataset.zip", "r") as zip_ref:
    zip_ref.extractall("dataset")


Saving dataset.zip to dataset.zip


In [ ]:
# Carrega modelo CLIP
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to("cpu")

def get_clip_features(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt").to("cpu")
    with torch.no_grad():
        features = model.get_image_features(**inputs)
    return features[0].numpy()

X, y = [], []
labels = ["cachorro", "pato", "passaro", "peixe"]
base_path = "dataset/"

for label in labels:
    for file in os.listdir(f"{base_path}/{label}"):
        path = os.path.join(base_path, label, file)
        if path.lower().endswith((".png", ".jpg", ".jpeg")):
            feat = get_clip_features(path)
            X.append(feat)
            y.append(label)

clf = LogisticRegression(max_iter=1000)
clf.fit(X, y)


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

LogisticRegression(max_iter=1000)

In [ ]:
joblib.dump(clf, "modelo_clip.pkl")


['modelo_clip.pkl']

In [ ]:
from google.colab import files
files.download("modelo_clip.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import joblib
clf = joblib.load("modelo_clip.pkl")


In [ ]:
# TESTAR MODELO
from PIL import Image
from google.colab import files

def prever_imagem(path_imagem):
    features = get_clip_features(path_imagem)  # Função que extrai embeddings com CLIP
    predicao = clf.predict([features])[0]
    return predicao

# Fazer upload de imagem
uploaded = files.upload()

for fn in uploaded.keys():
    # Abrir e redimensionar imagem
    img_path = fn
    print(prever_imagem(img_path))


Saving e2aec7483b3b88712c3390975d0bb0abc4767bdf.jpeg to e2aec7483b3b88712c3390975d0bb0abc4767bdf.jpeg
Saving 2914ef47ca79983c283736e1baf22a321c95b51b.jpeg to 2914ef47ca79983c283736e1baf22a321c95b51b.jpeg
Saving c41ee936aebb23f4bc8208c4239c13a415c75887.jpeg to c41ee936aebb23f4bc8208c4239c13a415c75887.jpeg
Saving 867f2477705ac69f22be267156a66d88f5ba078b.jpeg to 867f2477705ac69f22be267156a66d88f5ba078b.jpeg
Saving 364f43686c5c3443405861d95290b25a5522af9d.jpeg to 364f43686c5c3443405861d95290b25a5522af9d.jpeg
Saving dab0e62d084e96c8cc4665f551869c178a67657c.jpeg to dab0e62d084e96c8cc4665f551869c178a67657c.jpeg
Saving e8b9b67a0fa5888ea5e666cf3d5a57c9dcccbd40.jpeg to e8b9b67a0fa5888ea5e666cf3d5a57c9dcccbd40.jpeg
Saving 111d75bfef05ab8235ed2c23fa37bd096159b254.jpeg to 111d75bfef05ab8235ed2c23fa37bd096159b254.jpeg
Saving 34dc9c36bbb7161de240f874ca3345ff57e9c9cd.jpeg to 34dc9c36bbb7161de240f874ca3345ff57e9c9cd.jpeg
Saving 671c3fdc70e1adbcde4e672c2cc2cda451bb8f34.jpeg to 671c3fdc70e1adbcde4e672c2c